In [1]:
import os
import numpy as np
import pandas as pd
import model as m
from tqdm import tqdm
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.preprocessing.image import *

basedir = './imgs/'
df = pd.read_csv('driver_imgs_list.csv')
driver_list = list(df['subject'].unique())

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 分离训练集验证集

In [2]:
for i in range(5):
    m.train_valid_split(basedir, i+1, df, driver_list, driver_list[5*i: 5*i+3])

# ResNet50

In [2]:
from keras.applications.resnet50 import preprocess_input
from keras import backend as K
K.clear_session()

# 模型参数
model_image_size = (240, 320)
batch_size = 32
epoch = 1
model_name = ResNet50
fune_tune_layer = 0


# 图片生成器
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    channel_shift_range=10.)
gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator, steps_test_sample = m.test_gen(gen, basedir, model_image_size, batch_size)

Found 79726 images belonging to 1 classes.


In [4]:
# 模型训练和预测
df = pd.read_csv("sample_submission.csv")
results = np.zeros((79726, 10))
for i in range(1,6):
    train_generator, valid_generator, steps_train_sample, steps_valid_sample \
    = m.tv_gen(i, train_gen, gen, basedir, model_image_size, batch_size)
    
    model = m.build_model(model_name, model_image_size, fune_tune_layer)
    optimizer = Adam(lr=1e-4)
    m.train_model(model, optimizer, epoch, train_generator, valid_generator, steps_train_sample//2, steps_valid_sample)
    optimizer = Adam(lr=1e-5)
    m.train_model(model, optimizer, epoch, train_generator, valid_generator, steps_train_sample//2, steps_valid_sample)
    model.save('ResNet50_' + str(i) + '.h5')
    result = m.predict_model(model, test_generator, steps_test_sample)
    m.submission(df, result, 'ResNet50_' + str(i) + '.csv')
    results += result
results = results / 5.
m.submission(df, results, 'ResNet50merge.csv')

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.
Epoch 1/1
313/313 [==============================] - 441s 1s/step - loss: 0.3925 - acc: 0.8740 - val_loss: 0.6859 - val_acc: 0.7438
Epoch 1/1
2492/2492 [==============================] - 1085s 435ms/step
Found 18728 images belonging to 10 classes.
Found 3696 images belonging to 10 classes.
Epoch 1/1
293/293 [==============================] - 437s 1s/step - loss: 0.4368 - acc: 0.8643 - val_loss: 0.2395 - val_acc: 0.9340
Epoch 1/1
2492/2492 [==============================] - 1089s 437ms/step
Found 20577 images belonging to 10 classes.
Found 1847 images belonging to 10 classes.
Epoch 1/1
322/322 [==============================] - 452s 1s/step - loss: 0.3730 - acc: 0.8873 - val_loss: 0.3289 - val_acc: 0.9139
Epoch 1/1
2492/2492 [==============================] - 1092s 438ms/step
Found 19703 images belonging to 10 classes.
Found 2721 images belonging to 10 classes.
Epoch 1/1
308/308 [=====================

AttributeError: module 'model' has no attribute 'submission_clip'

# Xception

In [3]:
from keras.applications.xception import preprocess_input
from keras import backend as K
K.clear_session()

# 模型参数
model_image_size = (330, 440)
batch_size = 16
epoch = 1
model_name = Xception
fune_tune_layer = 0

# 图片生成器
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    channel_shift_range=10.)
gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator, steps_test_sample = m.test_gen(gen, basedir, model_image_size, batch_size)

Found 79726 images belonging to 1 classes.


In [4]:
# 模型训练和预测
df = pd.read_csv("sample_submission.csv")
results = np.zeros((79726, 10))
for i in range(1,6):
    train_generator, valid_generator, steps_train_sample, steps_valid_sample \
    = m.tv_gen(i, train_gen, gen, basedir, model_image_size, batch_size)
    
    model = m.build_model(model_name, model_image_size, fune_tune_layer)
    optimizer = Adam(lr=1e-4)
    m.train_model(model, optimizer, epoch, train_generator, valid_generator, steps_train_sample//2, steps_valid_sample)
    optimizer = Adam(lr=1e-5)
    m.train_model(model, optimizer, epoch, train_generator, valid_generator, steps_train_sample//2, steps_valid_sample)
    model.save('Xception' + str(i) + '.h5')
    result = m.predict_model(model, test_generator, steps_test_sample)
    m.submission(df, result, 'Xception' + str(i) + '.csv')
    results += result
results = results / 5.
m.submission(df, results, 'Xceptionmerge.csv')

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.
83689472/83683744 [==============================] - 1s 0us/step
Epoch 1/1
625/625 [==============================] - 1147s 2s/step - loss: 0.3527 - acc: 0.8993 - val_loss: 0.4078 - val_acc: 0.8601
Epoch 1/1
4983/4983 [==============================] - 2620s 526ms/step


100%|██████████| 79726/79726 [05:24<00:00, 245.43it/s]


Found 18728 images belonging to 10 classes.
Found 3696 images belonging to 10 classes.
Epoch 1/1
585/585 [==============================] - 1108s 2s/step - loss: 0.3693 - acc: 0.8956 - val_loss: 0.1697 - val_acc: 0.9523
Epoch 1/1
4983/4983 [==============================] - 2635s 529ms/step


100%|██████████| 79726/79726 [05:27<00:00, 243.20it/s]


Found 20577 images belonging to 10 classes.
Found 1847 images belonging to 10 classes.
Epoch 1/1
643/643 [==============================] - 1149s 2s/step - loss: 0.3482 - acc: 0.9013 - val_loss: 0.3003 - val_acc: 0.9047
Epoch 1/1
4983/4983 [==============================] - 2626s 527ms/step


100%|██████████| 79726/79726 [05:31<00:00, 240.44it/s]


Found 19703 images belonging to 10 classes.
Found 2721 images belonging to 10 classes.
Epoch 1/1
616/616 [==============================] - 1132s 2s/step - loss: 0.3438 - acc: 0.9027 - val_loss: 0.2942 - val_acc: 0.9225
Epoch 1/1
4983/4983 [==============================] - 2642s 530ms/step


100%|██████████| 79726/79726 [05:33<00:00, 239.01it/s]


Found 19761 images belonging to 10 classes.
Found 2663 images belonging to 10 classes.
Epoch 1/1
618/618 [==============================] - 1138s 2s/step - loss: 0.3459 - acc: 0.9019 - val_loss: 0.6162 - val_acc: 0.7923
Epoch 1/1
4983/4983 [==============================] - 2652s 532ms/step


100%|██████████| 79726/79726 [05:34<00:00, 238.59it/s]


# InceptionV3

In [2]:
from keras.applications.inception_v3 import preprocess_input
from keras import backend as K
K.clear_session()

# 模型参数
model_image_size = (330, 440)
batch_size = 32
epoch = 1
model_name = InceptionV3
fune_tune_layer = 0

# 图片生成器
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    channel_shift_range=10.)
gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator, steps_test_sample = m.test_gen(gen, basedir, model_image_size, batch_size)

Found 79726 images belonging to 1 classes.


In [ ]:
# 模型训练和预测
df = pd.read_csv("sample_submission.csv")
results = np.zeros((79726, 10))
for i in range(1,6):
    train_generator, valid_generator, steps_train_sample, steps_valid_sample \
    = m.tv_gen(i, train_gen, gen, basedir, model_image_size, batch_size)
    
    model = m.build_model(model_name, model_image_size, fune_tune_layer)
    optimizer = Adam(lr=1e-4)
    m.train_model(model, optimizer, epoch, train_generator, valid_generator, steps_train_sample//2, steps_valid_sample)
    optimizer = Adam(lr=1e-5)
    m.train_model(model, optimizer, epoch, train_generator, valid_generator, steps_train_sample//2, steps_valid_sample)
    model.save('InceptionV3_' + str(i) + '.h5')
    result = m.predict_model(model, test_generator, steps_test_sample)
    m.submission(df, result, 'InceptionV3_' + str(i) + '.csv')
    results += result

results = results / 5.
m.submission(df, results, 'InceptionV3merge.csv')